In [2]:
import pandas as pd
from sqlalchemy.orm import sessionmaker#,relationship, backref
from sqlalchemy import create_engine,select
#from Schema import *
from utils_modeling import *

In [ ]:
def get_metric_ts(metrc,idx,instrument):
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    sql = select([Metrics]).where(and_(Metrics.instrument_id == instrument,
        Metrics.Metric == metrc,Metrics.equity_fundamental_id == idx)).select_from(Metrics)
    result=s.execute(sql)
    df_ = pd.DataFrame(result.fetchall()).drop_duplicates()
    df_.columns = result.keys()
    s.close()
    return df_

def get_metrics(instrument):
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    sql = select([Metrics]).where(Metrics.instrument_id == instrument).select_from(Metrics)
    result=s.execute(sql)
    df_ = pd.DataFrame(result.fetchall()).drop_duplicates()
    df_.columns = result.keys()
    s.close()
    return df_

In [ ]:
df1=get_metric_ts("Open",2,379)
df2=get_metric_ts("Market Cap",1,379)
df3=get_metric_ts("Day's Range",2,379)
df4=get_metric_ts("Avg. Volume",2,379)
df5=get_metric_ts("1y Target Est",1,379)
df6=get_metric_ts("Enterprise Value 3",2,379)

df7=get_metric_ts("Trailing P/E",2,379)
df8=get_metric_ts("Forward P/E 1",2,379)
df9=get_metric_ts("No. of Analysts",8,379)
df10=get_metric_ts("Avg. Estimate",8,379)
df11=get_metric_ts("Low Estimate",8,379)
df12=get_metric_ts("High Estimate",8,379)
df13=get_metric_ts("Year Ago EPS",8,379)

In [ ]:
df4

In [ ]:
result=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13])
result=result[(result.Period!="Next Qtr. (Sep 2018)")&(result.Period!="Current Year (2018)")&(result.Period!="Next Year (2019)")]
result=result.sort_values(["refreshed_at","Metric"])

instrument=get_data(Instrument)[["id","name"]]
equity=get_data(Equity_fundamentals)

result=pd.merge(result,instrument,left_on="instrument_id",right_on="id")
result=pd.merge(result,equity,left_on="equity_fundamental_id",right_on="id")
result=result.sort_values(["refreshed_at","Metric"])
result=result[["name","Category","Table","Metric","Period","Value","refreshed_at"]]
result.columns=["Ticker","Category","Section","Metric","Period","Value","Date"]
#result.set_index("Date")
#result["Date2"]=pd.to_datetime(result['Date'])#, format='%Y-%m-%d %H:%M:%S')
result=result.sort_values(["Date"])

In [ ]:
#result['Date']

In [ ]:
#result["DATE2"]= ts.dt.date
#result["DATE2"]=pd.to_datetime(result['Date'], format="%Y-%m-%d")
result["DATE2"]=result['Date'].dt.date.astype(str)

In [ ]:
result.head()

In [ ]:
result[result.DATE2=="2018-05-20"]

In [ ]:
result.DATE2.drop_duplicates().shape

In [ ]:
#result.groupby(['DATE2']).count()
result.groupby(['Metric']).count()

In [ ]:
result.to_csv("SBUX.csv")

In [ ]:
all_data=get_metrics(379)

In [ ]:
all_data.head()
instrument=get_data(Instrument)[["id","name"]]
equity=get_data(Equity_fundamentals)

result=pd.merge(all_data,instrument,left_on="instrument_id",right_on="id")
result=pd.merge(result,equity,left_on="equity_fundamental_id",right_on="id")
result=result.sort_values(["refreshed_at","Metric"])
result=result[["name","Category","Table","Metric","Period","Value","refreshed_at"]]
result.columns=["Ticker","Category","Section","Metric","Period","Value","Date"]
#result.set_index("Date")
#result["Date2"]=pd.to_datetime(result['Date'])#, format='%Y-%m-%d %H:%M:%S')
result=result.sort_values(["Date"])
result.head()

In [ ]:
result.to_csv("SBUX.csv")

#  TEST DOWNLOAD OPTIONS INFO FROM YAHOO

In [ ]:
import sys
sys.path.append('/home/ubuntu/quantum/Infrastructure')
#from download_historical_equity_fundamental import *
from Schema import *
from mvo_utils import *
sys.path.append('home/ubuntu/quantum/Modeling')

In [ ]:
def get_summary(name):
    try:
        name1=name.replace('WIKI/' ,'')
        summaryf=get_data_sf(pd.read_html('https://finance.yahoo.com/quote/'+name1+'/options?p='+name1))[1]
        summaryf.columns=["Metric","Value"]
        summaryf["Category"]="Summary";summaryf["Table"]="Summary"
        summaryf["Period"]=str(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d'))
        summaryf["refreshed_at"]=datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        summaryf["instrument_id"]=get_id_instrument(name)
        summaryf["data_vendor_id"]=get_id_data_vendor("Yahoo_Finance")
        summaryf["equity_fundamental_id"]=summaryf[["Category","Table"]].apply(lambda x :add_up_equityf(x["Category"],x["Table"]), axis=1)
        summaryf[["instrument_id","equity_fundamental_id","data_vendor_id","refreshed_at","Period","Metric","Value"]].to_csv(Path_stats+"summary_"+name1+".csv")
        return True
    except Exception as err:
        print("Error: {0}".format(err))
        print("No se existe summary para "+name)
        return err
        pass
    
# https://finance.yahoo.com/quote/SBUX/options?p=SBUX
def get_data_sf(data_request):
    count=0
    while True:
        try:
            count=count+1
            summary = data_request ; break
            if count==limit:
                break;
        except Exception as err:
            print("Error: {0}, waiting to try again in {1}".format(err, sleep_time))
            sleep(sleep_time)
            if count==limit:
                break;
    return summary

In [ ]:
name="SBUX"

name1=name.replace('WIKI/' ,'')
summaryf=get_data_sf(pd.read_html("https://www.marketwatch.com/investing/stock/SBUX/options?countrycode=US&showAll=True"))[0]

In [ ]:
summaryf.columns=["var0","var1","var2","var3","var4","var5","var6","var7","var8","var9","var10","var11","var12","var13","var14"]
summaryf=summaryf[summaryf.var0.notnull()]
summaryf.index=range(summaryf.shape[0])

In [ ]:
#summaryf['var0']
summaryf.shape[0]

In [ ]:
idx_subtable=summaryf[summaryf['var0'].str.contains('Expires')].index.tolist()
idx_subtable.append(summaryf.shape[0])
idx_subtable

In [ ]:
lista=[[idx_subtable[i],idx_subtable[i+1]]for i in range(len(idx_subtable)-1)]
lista

In [ ]:
df_final=pd.DataFrame()
for range_ in lista:
    idx_0=range_[0]
    idx_1=range_[1]
    aux=summaryf.loc[idx_0]["var0"].replace('Expires ', '').replace(',', '')
    datetime_object = datetime.strptime(aux,'%B %d %Y').date()
    df_aux=summaryf.loc[idx_0+1:idx_1]#[summaryf.var1.notnull()]
    df_aux=df_aux[df_aux.var1.notnull()]
    df1=df_aux[["var1","var2","var3","var4","var5","var6","var7"]]
    df1["type"]='Call'
    df1.columns=["last","change","volume","bid","ask","open_interest","strike","type"]

    df2=df_aux[["var9","var10","var11","var12","var13","var14","var7"]]
    df2.columns=["last","change","volume","bid","ask","open_interest","strike"]
    df2["type"]="Put"

    df_final_aux=df1.append(df2)
    df_final_aux["expiration"]=datetime_object
    df_final_aux["date_"]=datetime.today().strftime("%Y-%m-%d")
    df_final=df_final.append(df_final_aux)

In [ ]:
df_final

In [ ]:
#summaryf.loc[4:54].head()

In [ ]:
aux=summaryf.loc[3]["var0"].replace('Expires ', '').replace(',', '')
from datetime import datetime
print(aux)
datetime_object = datetime.strptime(aux,'%B %d %Y').date()
datetime_object

In [ ]:
#df_aux=summaryf.loc[5:54][summaryf.var1.notnull()]
df_aux=summaryf.loc[5:54]#[summaryf.var1.notnull()]
df_aux=df_aux[df_aux.var1.notnull()]
df1=df_aux[["var1","var2","var3","var4","var5","var6","var7"]]
df1["type"]='Call'
df1.columns=["last","change","volume","bid","ask","open_interest","strike","type"]

df2=df_aux[["var9","var10","var11","var12","var13","var14","var7"]]
df2.columns=["last","change","volume","bid","ask","open_interest","strike"]
df2["type"]="Put"

df_final=df1.append(df2)
df_final["expiration"]=datetime_object
df_final["date_"]=datetime.today().strftime("%Y-%m-%d")
df_final

In [ ]:
call.head()

In [ ]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup as BS
import datetime
import pandas as pd

#url ="https://finance.yahoo.com/quote/SBUX/options?p=SBUX" #"https://www.barchart.com/stocks/quotes/SBUX/options"#
def get_list_dates(name):
    url = "https://query2.finance.yahoo.com/v7/finance/options/"+name
    usock = urlopen(url)
    data = usock.read()
    usock.close()
    soup = BS(data,"lxml")

    python_dict = json.loads(soup.getText())
    list_dates=python_dict['optionChain']['result'][0]['expirationDates']
    return list_dates

name="SBUX"

def get_options(name):
    call=pd.DataFrame()
    put=pd.DataFrame()
    name1=name.replace('WIKI/' ,'')
    list_dates=get_list_dates(name)
    #print(list_dates)
    for j in list_dates:
        #print(j)
        try:
            calls=pd.read_html("https://finance.yahoo.com/quote/"+name+"/options?p="+name+"&date="+str(j))[0]
            calls["expiration"]=datetime.datetime.fromtimestamp(int(j)).strftime('%Y-%m-%d')
            call=call.append(calls)
        except Exception as err:
            print("Error: {0}".format(err))
        try:
            puts=pd.read_html("https://finance.yahoo.com/quote/"+name+"/options?p="+name+"&date="+str(j))[1]
            puts["expiration"]=datetime.datetime.fromtimestamp(int(j)).strftime('%Y-%m-%d')
            put=put.append(puts)
        except Exception as err:
            print("Error: {0}".format(err))
    call["type"]="call"
    put["type"]="put"
    final=call.append(put)
    final["ticker"]=name
    return final

In [ ]:
options_sbux=get_options(name)

In [ ]:
options_sbux.head()

In [ ]:
options_sbux.to_csv("SBUX_options.csv")

# Analisis options SBUX

In [16]:
options_sbux

,Unnamed: 0,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,expiration,type,ticker,ITM,LT_WeekDay,LT_Hour,B/A Spread,B/A Spread %
0,0,SPY180711C00240000,2018-06-08 16:05:00,240.0,37.24,38.19,38.41,0.00,0.0000,2,0,2.1445,2018-07-11 00:00:00,call,SBUX,1,4,16,0.22,0.005761
1,1,SPY180711C00245000,2018-07-10 15:59:00,245.0,33.93,31.45,31.94,0.00,0.0000,7,0,0.9531,2018-07-11 00:00:00,call,SBUX,1,1,15,0.49,0.015580
2,2,SPY180711C00250000,2018-07-09 10:47:00,250.0,27.01,25.50,27.96,0.00,0.0000,5,5,1.5469,2018-07-11 00:00:00,call,SBUX,1,0,10,2.46,0.096471
3,3,SPY180711C00255000,2018-06-29 23:46:00,255.0,14.62,16.62,16.89,0.00,0.0000,1,1,0.0000,2018-07-11 00:00:00,call,SBUX,1,4,23,0.27,0.016245
4,4,SPY180711C00259000,2018-06-27 14:37:00,259.0,13.81,12.62,12.81,0.00,0.0000,1,0,0.0000,2018-07-11 00:00:00,call,SBUX,1,2,14,0.19,0.015055
5,5,SPY180711C00260000,2018-07-10 11:12:00,260.0,18.72,18.81,19.04,0.78,0.0435,6,48,1.3716,2018-07-11 00:00:00,call,SBUX,1,1,11,0.23,0.012228
6,6,SPY180711C00261000,2018-06-29 23:46:00,261.0,9.24,10.87,11.13,0.00,0.0000,2,2,0.0000,2018-07-11 00:00:00,call,SBUX,1,4,23,0.26,0.023919
7,7,SPY180711C00262000,2018-06-29 23:46:00,262.0,8.36,9.95,10.20,0.00,0.0000,2,2,0.0000,2018-07-11 00:00:00,call,SBUX,1,4,23,0.25,0.025126
8,8,SPY180711C00263000,2018-07-06 13:31:00,263.0,12.66,12.43,12.62,4.53,0.5572,10,105,0.0000,2018-07-11 00:00:00,call,SBUX,1,4,13,0.19,0.015286
9,9,SPY180711C00264000,2018-07-09 09:30:00,264.0,12.70,14.81,15.04,0.00,0.0000,2,153,1.1587,2018-07-11 00:00:00,call,SBUX,1,0,9,0.23,0.015530


In [85]:
from datetime import datetime
import numpy as np
#datetime.strptime(, '%Y-%m-%d %H:%M:%S')
#df['url'].replace({'icashier.alipay.com': 'aliexpress.com'}, inplace=True, regex=True)
options_sbux=pd.read_csv("SPY_options.csv")
#options_sbux['last_trade_date'] = options_sbux["Last Trade Date"].replace({'PM':' '}, inplace=True, regex=True)
#options_sbux['last_trade_date'] = options_sbux["Last Trade Date"].str.replace('PM', ' ').str.replace('AM', ' ')
#options_sbux['Date']=pd.to_datetime(options_sbux["last_trade_date"], format='%Y-%m-%d %H:%M:%S')
#options_sbux['Date']=options_sbux['Date'].dt.date
precio_actual=276.91#48.98
#options_sbux['ITM'] = np.where((options_sbux['Strike']>precio_actual), '1', '0')

aux_options=options_sbux[options_sbux.ITM==0]
#aux_options["days"]=aux_options["expiration"]-
aux_options["expiration"]=pd.to_datetime(aux_options['expiration'])#.date()
import datetime as dt
aux_options["days"]=(aux_options["expiration"]-pd.to_datetime("2018-07-11")).dt.days

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
#aux_options.head()

In [86]:
#pd.DataFrame({'sum' : options_sbux.groupby( [ "Date"] ).sum()})#.reset_index()
oi_total=pd.DataFrame(aux_options.groupby('expiration')['Open Interest'].sum())
oi_total.columns=["Total"]
oi_total.head()
oi_call=pd.DataFrame(aux_options[aux_options.type=="call"].groupby('expiration')['Open Interest'].sum())
oi_call.columns=["call"]
oi_call.head()
oi_put=pd.DataFrame(aux_options[aux_options.type=="put"].groupby('expiration')['Open Interest'].sum())
oi_put.columns=["put"]
final=pd.concat([oi_total,oi_call,oi_put],axis=1)
final.head()
final.sort_values(["Total"], ascending=False).head(5)

,Total,call,put
expiration,,,
2018-07-20,4126329,1412850,2713479
2018-09-21,2948036,1132771,1815265
2018-08-17,2469530,770423,1699107
2018-12-21,1654597,507805,1146792
2019-01-18,1172983,242771,930212


In [87]:
table = pd.pivot_table(aux_options, values='Open Interest', index=['expiration'],columns=['type'], aggfunc=np.sum)
table["total"]=table["call"]+table["put"]
table.sort_values(["total"], ascending=False).head(5)

type,call,put,total
expiration,,,
2018-07-20,1412850,2713479,4126329
2018-09-21,1132771,1815265,2948036
2018-08-17,770423,1699107,2469530
2018-12-21,507805,1146792,1654597
2019-01-18,242771,930212,1172983


In [75]:
aux_options=aux_options[aux_options.days>30]
oi_total=pd.DataFrame(aux_options.groupby('expiration')['Open Interest'].sum())
oi_total.columns=["Total"]
oi_total.head()
oi_call=pd.DataFrame(aux_options[aux_options.type=="call"].groupby('expiration')['Open Interest'].sum())
oi_call.columns=["call"]
oi_call.head()
oi_put=pd.DataFrame(aux_options[aux_options.type=="put"].groupby('expiration')['Open Interest'].sum())
oi_put.columns=["put"]
final=pd.concat([oi_total,oi_call,oi_put],axis=1)
final.head()
final.sort_values(["Total"], ascending=False).head(5)

,Total,call,put
expiration,,,
2018-09-21,2948036,1132771,1815265
2018-08-17,2469530,770423,1699107
2018-12-21,1654597,507805,1146792
2019-01-18,1172983,242771,930212
2019-12-20,574658,133311,441347


In [27]:
oi_total[oi_total.index=='2018-08-17 00:00:00']

,Total
expiration,
2018-08-17 00:00:00,2469530


,call
expiration,
2018-07-11 00:00:00,78454
2018-07-13 00:00:00,276785
2018-07-16 00:00:00,29901
2018-07-18 00:00:00,39030
2018-07-20 00:00:00,1412850


,Total,call,put
expiration,,,
2018-07-11 00:00:00,204232,78454,125778
2018-07-13 00:00:00,868957,276785,592172
2018-07-16 00:00:00,72746,29901,42845
2018-07-18 00:00:00,81642,39030,42612
2018-07-20 00:00:00,4126329,1412850,2713479


,Total,call,put
expiration,,,
2018-07-20 00:00:00,4126329,1412850,2713479
2018-09-21 00:00:00,2948036,1132771,1815265
2018-08-17 00:00:00,2469530,770423,1699107
2018-12-21 00:00:00,1654597,507805,1146792
2019-01-18 00:00:00,1172983,242771,930212
